# VacationPy 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import pprint

# Import API key
from api_keys import g_key

#configure gmaps
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
dfVacation = pd.read_csv("output_data/cities.csv")
dfVacation.dropna(inplace = True)
dfVacation.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,CityDate,Temp (C)
0,0,Meulaboh,4.14,96.13,300.29,85,73,1.15,ID,1594864638,27.14
1,1,Zvishavane,-20.33,30.07,281.73,81,97,5.19,ZW,1594864638,8.58
2,2,Bandarbeyla,9.49,50.81,300.91,60,10,11.92,SO,1594864638,27.76
3,3,Punta Arenas,-53.15,-70.92,273.15,94,6,2.10,CL,1594864638,0.00
4,4,Atar,20.52,-13.05,304.88,22,0,3.45,MR,1594864638,31.73


In [3]:
# set the latitude and longitude
locations = dfVacation[["Lat", "Lng"]]
humidity = dfVacation["Humidity"]
locations.head()

,Lat,Lng
0,4.14,96.13
1,-20.33,30.07
2,9.49,50.81
3,-53.15,-70.92
4,20.52,-13.05


In [4]:
# ### Humidity Heatmap
# * Configure gmaps.
# * Use the Lat and Lng as locations and Humidity as the weight.
# * Add Heatmap layer to map. 

In [5]:
#configure gmaps and 
figure_layout = {
    'width' : '400px',
    'height' : '300px',
    'border' : '1px solid black',
    'padding' : '1px',
    'margin' : '0 auto 0 auto'
}
fig = gmaps.figure(map_type = "HYBRID")
heatmap_layer = gmaps.heatmap_layer(locations,
                                    weights = humidity,
                                    dissipating = True,
                                    max_intensity = 200,
                                    point_radius = 10
                                   )
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.
Drop any rows will null values.

In [6]:
IdealWeather = dfVacation[(dfVacation["Temp (C)"] > 30) & (dfVacation["Temp (C)"] < 40) 
                           & (dfVacation["Humidity"] > 50)
                           & (dfVacation["Cloudiness"] < 40)
                           & (dfVacation["Wind Speed"] < 10)
                         ]
IdealWeather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,CityDate,Temp (C)
136,136,Wanning,18.80,110.40,304.22,69,8,6.27,CN,1594864647,31.07
228,228,Cabo San Lucas,22.89,-109.91,304.26,55,5,5.10,MX,1594864372,31.11
373,373,Sharjah city,25.36,55.40,309.15,59,0,2.60,AE,1594864394,36.00
461,461,Dickinson,29.46,-95.05,303.71,70,20,6.20,US,1594864501,30.56


### Hotel Map

Store into variable named hotel_df.

Add a "Hotel Name" column to the DataFrame.

Set parameters to search for hotels with 5000 meters.

Hit the Google Places API for each city's coordinates.

Store the first Hotel result into the DataFrame.

Plot markers on top of the heatmap.

In [11]:
hotelDF = IdealWeather
IdealWeather["Hotel Name"] = ""
IdealWeather.head()
IdealWeather.drop(["Max Temp"], axis = 1)

C:\Users\Ryan\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City_ID,City,Lat,Lng,Humidity,Cloudiness,Wind Speed,Country,CityDate,Temp (C),Hotel Name
136,136,Wanning,18.80,110.40,69,8,6.27,CN,1594864647,31.07,
228,228,Cabo San Lucas,22.89,-109.91,55,5,5.10,MX,1594864372,31.11,
373,373,Sharjah city,25.36,55.40,59,0,2.60,AE,1594864394,36.00,
461,461,Dickinson,29.46,-95.05,70,20,6.20,US,1594864501,30.56,


In [15]:
#setting for loop to send out multiple API requests
for index, row in IdealWeather.iterrows(): 
    vacLat = row["Lat"]
    vacLng = row["Lng"]
    vacCity = row["City"]
    nearbyUrl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
            "location" : f'{vacLat}, {vacLng}',
            "key" : g_key,
            "radius" : 5000,
            "type" : "lodging",
            "rankBy" : "prominence"
            }
# storing JSON object into a variable and then printing it into a dataframe
    response = requests.get(nearbyUrl, params)   
    pp = pprint.PrettyPrinter(indent=4)
    myHotel = response.json()
# setting try/except for any cities that do not meet the critera
    try:
        closeHotel = myHotel['results'][0]['name']
    except IndexError:
        pass
        IdealWeather.loc[index,"Hotel Name"] = closeHotel
    
IdealWeather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,CityDate,Temp (C),Hotel Name
136,136,Wanning,18.80,110.40,304.22,69,8,6.27,CN,1594864647,31.07,Paradise Hotel
228,228,Cabo San Lucas,22.89,-109.91,304.26,55,5,5.10,MX,1594864372,31.11,Hotel Tesoro Los Cabos
373,373,Sharjah city,25.36,55.40,309.15,59,0,2.60,AE,1594864394,36.00,"Radisson Blu Resort, Sharjah-United Arab Emirates"
461,461,Dickinson,29.46,-95.05,303.71,70,20,6.20,US,1594864501,30.56,Candlewood Suites League City


In [16]:
gmaps.marker_layer

<function gmaps.marker.marker_layer(locations, hover_text='', label='', info_box_content=None, display_info_box=None)>

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in IdealWeather.iterrows()]
locations = IdealWeather[["Lat", "Lng"]]
locations

,Lat,Lng
136,18.80,110.40
228,22.89,-109.91
373,25.36,55.40
461,29.46,-95.05


In [18]:
#adding locations to the heatmap figure
points_of_interest = gmaps.marker_layer(locations, info_box_content=hotel_info) 
fig.add_layer(points_of_interest)
fig

Figure(layout=FigureLayout(height='420px'))